In [ ]:
import h5py
from pathlib import Path
import numpy as np 
import matplotlib.pyplot as plt
import mediapy as media
import pickle
import sys
#You may have to change this path to work with your rPPG-toolbox path
sys.path.append("/homes/bacharya/rPPG-Toolbox/")
from evaluation.post_process import calculate_HR
from scipy.sparse import spdiags
from scipy.signal import butter
import scipy
import heartpy
from unsupervised_methods.unsupervised_predictor import ecg_processing

In [ ]:
paths = Path("/data/dst_tsst_22_bi_multi_nt_lab/processed/rPPG/DST_SizeW72_SizeH72_ClipLength180_DataTypeRaw_DataAugNone_LabelTypeRaw_Crop_faceTrue_Large_boxTrue_Large_size1.5_Dyamic_DetTrue_det_len1_Median_face_boxFalse_unsupervised/")

In [ ]:
paths = list(paths.rglob("*input*.npy"))

In [ ]:
len(paths)

In [ ]:
import numpy as np
import cv2
size = [72, 72]
fps = 30
#save the video created from numpy files ( maybe theres a more direct way to plot them ?) 
for path in paths:
    name = path.stem
    print(name)
    out = cv2.VideoWriter(f'{name}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), True)
    data = np.uint8( np.load(path))
    for i in range(data.shape[0]):
        out.write(data[i,:,:,::-1])
    out.release()

In [ ]:
from IPython.display import Video, display
from ipywidgets import Output, GridspecLayout

videoPaths = list(Path(".").rglob("*input0.mp4"))
videos = []
for path in videoPaths:
    videos.append(media.read_video(path))
media.show_videos(videos)

Most videos are good. However, there are a few exceptions that exist and would be interesting to look at these individual videos and see what the performance of the methods. 
The background for some are different even though they are cropped.
TODO: plotting the signal with the video makes sense to see the changes.

In [ ]:


def _reform_data_from_dict(data, flatten=True):
    """Helper func for calculate metrics: reformat predictions and labels from dicts. """
    # sort_data = sorted(data.items(), key=lambda x: x[0])
    # sort_data = [i[1] for i in sort_data]
    # sort_data = torch.cat(sort_data, dim=0)

    if flatten:
        sort_data = np.reshape(data, (-1))
    else:
        sort_data = np.array(data)

    return sort_data

def _process_signal(signal, fs=30, diff_flag=True):
    # Detrend and filter
    use_bandpass = True
    if diff_flag:  # if the predictions and labels are 1st derivative of PPG signal.
        gt_bvp = _detrend(np.cumsum(signal), 100)
    else:
        gt_bvp = _detrend(signal, 100)
    if use_bandpass:
        # bandpass filter between [0.75, 2.5] Hz
        # equals [45, 150] beats per min
        [b, a] = butter(1, [0.75 / fs * 2, 2.5 / fs * 2], btype='bandpass')
        signal = scipy.signal.filtfilt(b, a, np.double(signal))
    return signal

def _detrend(input_signal, lambda_value):
    """Detrend PPG signal."""
    signal_length = input_signal.shape[0]
    # observation matrix
    H = np.identity(signal_length)
    ones = np.ones(signal_length)
    minus_twos = -2 * np.ones(signal_length)
    diags_data = np.array([ones, minus_twos, ones])
    diags_index = np.array([0, 1, 2])
    D = spdiags(diags_data, diags_index,
                (signal_length - 2), signal_length).toarray()
    detrended_signal = np.dot(
        (H - np.linalg.inv(H + (lambda_value ** 2) * np.dot(D.T, D))), input_signal)
    return detrended_signal

In [ ]:
#Plotting the signal
data_out_path = "/data/dst_tsst_22_bi_multi_nt_lab/processed/rPPG/DST_SizeW72_SizeH72_ClipLength180_DataTypeRaw_DataAugNone_LabelTypeRaw_Crop_faceTrue_Large_boxTrue_Large_size1.5_Dyamic_DetTrue_det_len1_Median_face_boxFalse_unsupervised/CHROM_outputs.pickle"  # Output Data Path 
trial_idx = 0
chunk_size = 1800 # size of chunk to visualize: -1 will plot the entire signal
chunk_num = 0

# Read in data and list subjects
with open(data_out_path, 'rb') as f:
    # print(f.readlines())
    data = pickle.load(f)
    
# List of all video trials
trial_list = list(data['predictions'].keys())
print('Num Trials', len(trial_list))

In [ ]:
#plot all the figures
for i in range(len(trial_list)):
    trial_idx = i
    participant_id = trial_list[trial_idx]
    
    prediction = np.array(data['predictions'][trial_list[trial_idx]])
    label = np.array(data['labels'][trial_list[trial_idx]])
    
    # prediction = np.array(data['predictions'][trial_idx])
    
    # Read in meta-data from pickle file
    # fs = data['fs'] # Video Frame Rate
    fs = 25
    label_type = data['label_type'] # PPG Signal Transformation: `DiffNormalized` or `Standardized`
    diff_flag = (label_type == 'DiffNormalized')
    print(fs)
    if chunk_size == -1:
        chunk_size = len(prediction)
        chunk_num = 0

    label_frame_size = len(label)
    if label_frame_size>25000 and label_frame_size<30000:
        sampling_rate = 300
    else:
        sampling_rate = 1000
    gt_hr = ecg_processing(label, sampling_rate, plot_hp=True)
    print("Pred HR",calculate_HR(prediction, fs=fs))
    print("GT HR", gt_hr)
    print(participant_id, i )
    
    #process videos and visualize them
    path = f'./' + f'{participant_id}_input0.mp4' 
    video = media.read_video(path)
    # print("Number of frames", video.shape[0])
    print("Length of video", video.shape[0]/fs)
    media.show_video(video)
    # Process label and prediction signals
    prediction = _process_signal(prediction, fs, diff_flag=diff_flag)
    # label = _process_signal(label, fs, diff_flag=diff_flag)
    # start = (chunk_num)*chunk_size
    # stop = (chunk_num+1)*chunk_size
    samples = len(prediction)
    x_time = np.linspace(0, samples/fs, num=samples)
    
    plt.figure(figsize=(10,5))
    plt.plot(x_time, prediction, color='r')
    plt.title('Trial: ' + trial_list[trial_idx])
    plt.legend(['Predictions', 'Labels'])
    plt.xlabel('Time (s)');
    plt.show()


In [ ]:
%matplotlib inline

In [ ]:
#Interactive video viewer

from ipywidgets import interactive


trail_idx = 0
participant_id = trial_list[trial_idx]
prediction = np.array(data['predictions'][trial_list[trial_idx]])
diff_flag = (label_type == 'DiffNormalized')
path = f'./' + f'{participant_id}_input0.mp4' 
video = media.read_video(path)

def f(time):
    if time <=4:
        x = x_time[0:8*fs]
        pred = prediction[0:8*fs]
        vid = video[0:8*fs]
    else:
        x = x_time[(time-4)*fs:(time+4)*fs]
        pred = prediction[(time-4)*fs:(time+4)*fs]
        vid = video[(time-4)*fs:(time+4)*fs]
    
    plt.figure(figsize=(10,5))
    plt.plot(x, pred, color='r')
    # plt.title('Trial: ' + trial_list[trial_idx])
    plt.legend(['Predictions', 'Labels'])
    plt.xlabel('Time (s)')
    media.show_video(vid)

interactive_plot = interactive(f, time=(0, len(prediction)//fs - 2))
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot